### Setup

In [1]:
import pprint
from os import environ

import pyalex
import pandas as pd
from sqlalchemy import create_engine


In [2]:
# setup
pyalex.config.email = environ.get("OPENALEX_CONTACT_EMAIL")
# db
db_url = f"postgresql+psycopg://{environ.get('DB_USER')}:{environ.get('DB_PASSWORD')}@{environ.get('DB_HOST')}:5432/{environ.get('DB_NAME')}"
engine = create_engine(db_url)

In [3]:
# function for fetching openalex entities, returning dataframe
def fetch_all_objects(entity_type: pyalex.api.BaseOpenAlex, fields: list[str]) -> pd.DataFrame:
    from itertools import chain
    results = []
    for result in chain(*entity_type.select(fields).sort(id="asc").paginate(per_page=200)):
        results.append(result)
    return pd.DataFrame(results)

In [4]:
domains = fetch_all_objects(pyalex.Domains(), ["id", "display_name", "description", "ids", "updated_date"])
fields = fetch_all_objects(pyalex.Fields(), ["id", "display_name", "description", "ids", "updated_date", "domain"])
subfields = fetch_all_objects(pyalex.Subfields(), ["id", "display_name", "description", "ids", "updated_date", "field"])
topics = fetch_all_objects(pyalex.Topics(), ["id", "display_name", "description", "ids", "updated_date", "subfield", "keywords"])

In [5]:
topics # as provided by the API

,id,display_name,description,ids,updated_date,subfield,keywords
0,https://openalex.org/T10001,Tectonic and Geochronological Evolution of Oro...,This cluster of papers focuses on the tectonic...,"{'openalex': 'https://openalex.org/T10001', 'w...",2024-04-15T05:08:59.270713,"{'id': 'https://openalex.org/subfields/1908', ...","[Zircon, Geochronology, Tectonics, Granitic Ro..."
1,https://openalex.org/T10002,Advancements in Density Functional Theory,This cluster of papers represents advancements...,"{'openalex': 'https://openalex.org/T10002', 'w...",2024-04-15T05:11:05.125350,"{'id': 'https://openalex.org/subfields/3107', ...","[Density Functional Theory, Dispersion Correct..."
2,https://openalex.org/T10003,Knowledge Management and Organizational Innova...,This cluster of papers revolves around the top...,"{'openalex': 'https://openalex.org/T10003', 'w...",2024-04-15T05:09:54.770680,"{'id': 'https://openalex.org/subfields/1408', ...","[Dynamic Capabilities, Knowledge Transfer, Bus..."
3,https://openalex.org/T10004,Soil Carbon Dynamics and Nutrient Cycling in E...,This cluster of papers explores the dynamics o...,"{'openalex': 'https://openalex.org/T10004', 'w...",2024-04-15T05:18:28.084257,"{'id': 'https://openalex.org/subfields/1111', ...","[Soil Carbon Sequestration, Nitrogen Cycle, Mi..."
4,https://openalex.org/T10005,Biodiversity Conservation and Ecosystem Manage...,This cluster of papers focuses on the conserva...,"{'openalex': 'https://openalex.org/T10005', 'w...",2024-04-15T05:18:22.006411,"{'id': 'https://openalex.org/subfields/2309', ...","[Biodiversity, Conservation, Ecosystem, Invasi..."
...,...,...,...,...,...,...,...
4511,https://openalex.org/T14517,History of Science and Knowledge Production,This cluster of papers covers a wide range of ...,"{'openalex': 'https://openalex.org/T14517', 'w...",2024-04-15T05:18:07.300900,"{'id': 'https://openalex.org/subfields/1207', ...","[History, Science, Knowledge Production, Medic..."
4512,https://openalex.org/T14518,Baseball's Influence on American Culture and S...,This cluster of papers explores the profound i...,"{'openalex': 'https://openalex.org/T14518', 'w...",2024-04-15T05:13:01.425558,"{'id': 'https://openalex.org/subfields/1202', ...","[Baseball, American Culture, Societal Influenc..."
4513,https://openalex.org/T14519,Digital Education and Knowledge Economy,This cluster of papers explores the intersecti...,"{'openalex': 'https://openalex.org/T14519', 'w...",2024-04-15T05:06:52.448052,"{'id': 'https://openalex.org/subfields/1710', ...","[Digital Education, E-Learning, Knowledge Econ..."
4514,https://openalex.org/T14520,Slow Cities Movement and Sustainable Urban Dev...,This cluster of papers explores the Slow Citie...,"{'openalex': 'https://openalex.org/T14520', 'w...",2024-04-15T05:09:13.838428,"{'id': 'https://openalex.org/subfields/3322', ...","[Slow Cities, Sustainable Development, Urban S..."


### Cleanup and preprocessing for embedding and db
1. Extract numeric id from OpenAlex URL.
2. Create separate columns for wikipedia and wikidata URLs.
3. Add column with parent id (Domain -> Field -> Subfield -> Topic)

In [6]:
# function for preprocessing dataframe for embedding
def preprocess(df: pd.DataFrame, type_name: str) -> pd.DataFrame:
    df = df.copy()
    # id needs to be extracted from id column, which contains openalex urls
    id_regex = r"(\d+)$" # id is the number at the end of the url
    df["id"] = df["id"].str.extract(id_regex)
    df.set_index("id", verify_integrity=True, inplace=True)
    # create separate columns for wikidata and wikipedia urls from ids column
    df["wikipedia"] = df["ids"].apply(lambda x: x.get("wikipedia", None))
    # topics don't have wikidata urls
    if type_name != "topic":
        df["wikidata"] = df["ids"].apply(lambda x: x.get("wikidata", None))
    df.drop("ids", axis=1, inplace=True)
    
    
    type_to_parent_type = {
        "field": "domain",
        "subfield": "field",
        "topic": "subfield"
    }
    # extract parent id
    if type_name != "domain": # domains have no parent
        # extract parent url from dict
        df[type_to_parent_type[type_name]] = df[type_to_parent_type[type_name]].apply(lambda x: x.get("id", None))
        # and extract the id
        df[type_to_parent_type[type_name]] = df[type_to_parent_type[type_name]].str.extract(id_regex)
    
    return df

In [7]:
preprocessed_domains = preprocess(domains, "domain")
preprocessed_fields = preprocess(fields, "field")
preprocessed_subfields = preprocess(subfields, "subfield")
preprocessed_topics = preprocess(topics, "topic")

In [8]:
preprocessed_topics

,display_name,description,updated_date,subfield,keywords,wikipedia
id,,,,,,
10001,Tectonic and Geochronological Evolution of Oro...,This cluster of papers focuses on the tectonic...,2024-04-15T05:08:59.270713,1908,"[Zircon, Geochronology, Tectonics, Granitic Ro...",https://en.wikipedia.org/wiki/Geochronology
10002,Advancements in Density Functional Theory,This cluster of papers represents advancements...,2024-04-15T05:11:05.125350,3107,"[Density Functional Theory, Dispersion Correct...",https://en.wikipedia.org/wiki/Density_function...
10003,Knowledge Management and Organizational Innova...,This cluster of papers revolves around the top...,2024-04-15T05:09:54.770680,1408,"[Dynamic Capabilities, Knowledge Transfer, Bus...",https://en.wikipedia.org/wiki/Knowledge_manage...
10004,Soil Carbon Dynamics and Nutrient Cycling in E...,This cluster of papers explores the dynamics o...,2024-04-15T05:18:28.084257,1111,"[Soil Carbon Sequestration, Nitrogen Cycle, Mi...",https://en.wikipedia.org/wiki/Soil_carbon_dyna...
10005,Biodiversity Conservation and Ecosystem Manage...,This cluster of papers focuses on the conserva...,2024-04-15T05:18:22.006411,2309,"[Biodiversity, Conservation, Ecosystem, Invasi...",https://en.wikipedia.org/wiki/Biodiversity_con...
...,...,...,...,...,...,...
14517,History of Science and Knowledge Production,This cluster of papers covers a wide range of ...,2024-04-15T05:18:07.300900,1207,"[History, Science, Knowledge Production, Medic...",https://en.wikipedia.org/wiki/History_of_science
14518,Baseball's Influence on American Culture and S...,This cluster of papers explores the profound i...,2024-04-15T05:13:01.425558,1202,"[Baseball, American Culture, Societal Influenc...",https://en.wikipedia.org/wiki/Baseball_in_the_...
14519,Digital Education and Knowledge Economy,This cluster of papers explores the intersecti...,2024-04-15T05:06:52.448052,1710,"[Digital Education, E-Learning, Knowledge Econ...",https://en.wikipedia.org/wiki/Digital_education


### Generate string representation used as input for embedding
Essentially, concatenate column names with their content:

`"Topic: {topic_name}; Description: {topic_description}; Keywords: {topic_keywords}"`

In [9]:
# function to generate the string representation for embedding
def generate_string_representation_for_embedding(df: pd.DataFrame, column_prefix_map: dict) -> pd.Series:
    # first, check if all columns are present in the dataframe
    for column in column_prefix_map.keys():
        if column not in df.columns:
            raise ValueError(f"Column {column} not found in the dataframe.")
    
    # Initialize a list to store the concatenated strings for each row
    concatenated_strings = pd.Series(dtype=str)
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Initialize an empty list to store the strings for the current row
        row_strings = []
        # Iterate over the column map dictionary
        for column_name, prefix in column_prefix_map.items():
            column_value = row[column_name]
            if type(column_value) == list:
                column_value = ", ".join(column_value)
            # Concatenate the column's value for the current row with the corresponding string
            row_strings.append(prefix + ": " + str(column_value).strip())
        # Join the strings for the current row with "; " and add to the list
        concatenated_strings.at[index] ="; ".join(row_strings)
        
    return concatenated_strings

In [10]:
preprocessed_domains["string_representation"] = generate_string_representation_for_embedding(preprocessed_domains, {
    "display_name": "Domain",
    "description": "Description"
})
preprocessed_fields["string_representation"] = generate_string_representation_for_embedding(preprocessed_fields, {
    "display_name": "Field",
    "description": "Description"
})
preprocessed_subfields["string_representation"] = generate_string_representation_for_embedding(preprocessed_subfields, {
    "display_name": "Subfield",
    "description": "Description"
})
preprocessed_topics["string_representation"] = generate_string_representation_for_embedding(preprocessed_topics, {
    "display_name": "Topic",
    "description": "Description",
    "keywords": "Keywords"
})

In [11]:
preprocessed_topics["string_representation"].iloc[0]

'Topic: Tectonic and Geochronological Evolution of Orogens; Description: This cluster of papers focuses on the tectonic evolution and geochronological analysis of orogenic processes, with a particular emphasis on zircon geochemistry, isotopic composition, and thermodynamic modeling. It covers topics such as subduction zones, mantle evolution, plate tectonics, and the growth of continental crust.; Keywords: Zircon, Geochronology, Tectonics, Granitic Rocks, Isotopic Composition, Subduction Zones, Mantle Evolution, Plate Tectonics, Thermodynamic Modeling, Continental Growth'

### Create embeddings

In [12]:
# function to calculate sum of tokens for a Series
import tiktoken
def calculate_total_tokens(series: pd.Series, encoding_name: str = "cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return series.apply(lambda x: len(encoding.encode(x))).sum()
# function to calculate embedding costs for a count of tokens
def calculate_embedding_costs(num_tokens: int, price_per_m_tokens: float = 0.13) -> float:
    price_per_token = price_per_m_tokens / 1000000
    return num_tokens * price_per_token
# TODO: function to calculate embedding costs for a Series

In [13]:
total_tokens_for_domains = calculate_total_tokens(preprocessed_domains["string_representation"])
total_tokens_for_fields = calculate_total_tokens(preprocessed_fields["string_representation"])
total_tokens_for_subfields = calculate_total_tokens(preprocessed_subfields["string_representation"])
total_tokens_for_topics = calculate_total_tokens(preprocessed_topics["string_representation"])

embedding_costs_for_domains = calculate_embedding_costs(total_tokens_for_domains)
embedding_costs_for_fields = calculate_embedding_costs(total_tokens_for_fields)
embedding_costs_for_subfields = calculate_embedding_costs(total_tokens_for_subfields)
embedding_costs_for_topics = calculate_embedding_costs(total_tokens_for_topics)

print(f"Total tokens for Domains: {total_tokens_for_domains} - Embedding costs: {embedding_costs_for_domains}")
print(f"Total tokens for Fields: {total_tokens_for_fields} - Embedding costs: {embedding_costs_for_fields}")
print(f"Total tokens for Subfields: {total_tokens_for_subfields} - Embedding costs: {embedding_costs_for_subfields}")
print(f"Total tokens for Topics: {total_tokens_for_topics} - Embedding costs: {embedding_costs_for_topics}")
print(f"Total embedding costs: {embedding_costs_for_domains + embedding_costs_for_fields + embedding_costs_for_subfields + embedding_costs_for_topics}")

Total tokens for Domains: 92 - Embedding costs: 1.196e-05
Total tokens for Fields: 559 - Embedding costs: 7.267e-05
Total tokens for Subfields: 5092 - Embedding costs: 0.00066196
Total tokens for Topics: 550122 - Embedding costs: 0.07151586
Total embedding costs: 0.07226245


In [14]:
from openai import OpenAI
# function to get embeddings via OpenAI-API. Max. input is 2048 strings
def get_embeddings(input: pd.Series, batch_size: int = 2048, model="text-embedding-3-large", dimensions=1024) -> pd.Series:
    client = OpenAI() # requires OPENAI_API_KEY environment variable to be set
    # batch requests to speed up API calls
    embeddings = []
    for i in range(0, len(input), batch_size):
        batch = input.iloc[i:i+batch_size]
        response = client.embeddings.create(input=batch.tolist(), model=model, dimensions=dimensions)
        embeddings.extend([r.embedding for r in response.data])
    return pd.Series(embeddings, index=input.index)

def get_embedding(text: str, model="text-embedding-3-large", dimensions=1024):
    client = OpenAI() # requires OPENAI_API_KEY environment variable to be set
    response = client.embeddings.create(input=[text], model=model, dimensions=dimensions)
    return response.data[0].embedding

In [15]:
preprocessed_domains["embedding"] = get_embeddings(preprocessed_domains["string_representation"])
preprocessed_fields["embedding"] = get_embeddings(preprocessed_fields["string_representation"])
preprocessed_subfields["embedding"] = get_embeddings(preprocessed_subfields["string_representation"])
preprocessed_topics["embedding"] = get_embeddings(preprocessed_topics["string_representation"])

In [16]:
preprocessed_topics.iloc[0]

display_name             Tectonic and Geochronological Evolution of Oro...
description              This cluster of papers focuses on the tectonic...
updated_date                                    2024-04-15T05:08:59.270713
subfield                                                              1908
keywords                 [Zircon, Geochronology, Tectonics, Granitic Ro...
wikipedia                      https://en.wikipedia.org/wiki/Geochronology
string_representation    Topic: Tectonic and Geochronological Evolution...
embedding                [-0.014522067271173, -0.005941309034824371, -0...
Name: 10001, dtype: object

### Insert into database and cosine similarity query test

In [17]:
# function to prepare dataframe for database insertion
def prepare_dataframe_for_db(df: pd.DataFrame) -> pd.DataFrame:
    df_for_db = df.copy()
    df_for_db.drop(["string_representation"], axis=1, inplace=True)
    df_for_db.rename(columns={"display_name": "name", "domain": "domain_id", "field": "field_id", "subfield": "subfield_id"}, inplace=True)
    return df_for_db

In [18]:
domains_for_db = prepare_dataframe_for_db(preprocessed_domains)
fields_for_db = prepare_dataframe_for_db(preprocessed_fields)
subfields_for_db = prepare_dataframe_for_db(preprocessed_subfields)
topics_for_db = prepare_dataframe_for_db(preprocessed_topics)

In [19]:
from sqlalchemy import String, DateTime, Integer, ARRAY
from pgvector.sqlalchemy import Vector

# function to insert dataframe into database
def insert_dataframe_into_db(df: pd.DataFrame, type_name: str):
    type_to_parent_type = {
        "field": "domain",
        "subfield": "field",
        "topic": "subfield"
    }
    
    column_name_to_column_type = {
        "id": Integer,
        "name": String,
        'description': String,
        "updated_date": DateTime,
        "wikipedia": String,
        "embedding": Vector(1024)
    }
    
    if type_name != "topic":
        column_name_to_column_type["wikidata"] = String
    else:
        column_name_to_column_type["keywords"] = ARRAY(String, dimensions=1)
        
    if type_name != "domain":
        column_name_to_column_type[type_to_parent_type[type_name] + "_id"] = Integer
    
    table_name = f"openalex_{type_name}"
    
    
    df.to_sql(table_name, con=engine, if_exists="append", dtype=column_name_to_column_type)

In [21]:
insert_dataframe_into_db(domains_for_db, "domain")
insert_dataframe_into_db(fields_for_db, "field")
insert_dataframe_into_db(subfields_for_db, "subfield")
insert_dataframe_into_db(topics_for_db, "topic")

In [23]:
# get embeddings for a test query
query = "Leveraging LLMs for Personalized Suggestion and Summarization of Scientific Publications"
query_embedding = get_embedding(query)

### Test

In [24]:
# calculate cosine similarity for test query via db
from sqlalchemy import select, desc, text
from sqlalchemy.orm import Session
from db.models import Domain, Topic, Subfield

with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    statement = select(Topic, (1 - Topic.embedding.cosine_distance(query_embedding)).label('cosine_similarity')).order_by(desc('cosine_similarity')).limit(5)
    results = session.execute(statement).fetchall()
    
for result in results:
    print(f"{result.Topic.name}: {result.cosine_similarity}")

Recommender System Technologies: 0.5040306603954247
Authorship Attribution and User Profiling in Text: 0.45423276858828343
Automatic Text Simplification and Readability Assessment: 0.45315609732735473
Natural Language Processing: 0.4506736666365663
Optimization Methods in Machine Learning: 0.4344742427609637
